In [ ]:
import gradio as gr
import os
import json
import numpy as np
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
import faiss

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Load models
llm = ChatGroq(model="llama-3.1-8b-instant")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize FAISS index
d = 384  # MiniLM embedding dimension
if os.path.exists("user_history.index"):
    index = faiss.read_index("user_history.index")
else:
    index = faiss.IndexFlatL2(d)

# Load user history
def load_user_history():
    if os.path.exists("user_history.json"):
        with open("user_history.json", "r") as file:
            return json.load(file)
    return []

# Store user history along with feedback
def store_user_history(data, score, feedback):
    history = load_user_history()
    entry = {"vitals": data, "score": score, "feedback": feedback}
    history.append(entry)
    with open("user_history.json", "w") as file:
        json.dump(history, file, indent=4)
    
    # Append feedback to feedback history file
    if feedback:
        with open("feedback.txt", "a") as file:
            file.write(f"{feedback}\n")
    
    # Update embeddings in FAISS index
    embedding = embedding_model.encode(json.dumps(entry)).tolist()
    index.add(np.array([embedding], dtype=np.float32))
    faiss.write_index(index, "user_history.index")

# Retrieve similar cases using FAISS index
def retrieve_similar_cases(data):
    if index.ntotal == 0:
        return []
    embedding = embedding_model.encode(json.dumps(data)).tolist()
    distances, indices = index.search(np.array([embedding], dtype=np.float32), k=3)
    history = load_user_history()
    return [history[i] for i in indices[0] if i < len(history)]

# Predict recovery score based on vitals
def predict_recovery_score(data):
    prompt = f"""
    Given vitals:
    - Heart Rate: {data['heart_rate']}
    - BP: {data['bp']}
    - Oxygen Saturation: {data['oxygen_saturation']}
    - Symptoms: {data['symptoms']}
    Predict a recovery score between 0-100. Only give a score, no description.
    """
    return llm.predict(prompt).strip()

# Generate recovery plan using feedback history and similar cases
def generate_recovery_plan(score, feedback_history, user_history, similar_cases):
    prompt = f"""
    The patient's recovery score is {score}. Suggest a structured recovery plan.
    Consider the following:
    - Feedback History: {feedback_history}
    - User History: {user_history}
    - Similar Cases: {similar_cases}
    Adjust the plan to address any concerns or requests raised in the feedback.
    """
    return llm.predict(prompt).strip()

# Get feedback history
def get_feedback_history():
    if os.path.exists("feedback.txt"):
        with open("feedback.txt", "r") as file:
            return file.read()
    return "No feedback yet."

# Process user input to generate recovery score and plan
def process_input(heart_rate, bp, oxygen_saturation, symptoms):
    data = {
        "heart_rate": heart_rate,
        "bp": bp,
        "oxygen_saturation": oxygen_saturation,
        "symptoms": symptoms,
    }
    score = predict_recovery_score(data)
    feedback_history = get_feedback_history()
    user_history = load_user_history()
    similar_cases = retrieve_similar_cases(data)
    plan = generate_recovery_plan(score, feedback_history, user_history, similar_cases)
    return score, plan, "Please provide feedback on the generated recovery plan."

# Process feedback and store it
def process_feedback(heart_rate, bp, oxygen_saturation, symptoms, feedback):
    data = {
        "heart_rate": heart_rate,
        "bp": bp,
        "oxygen_saturation": oxygen_saturation,
        "symptoms": symptoms,
    }
    score = predict_recovery_score(data)
    store_user_history(data, score, feedback)
    feedback_history = get_feedback_history()
    return f"Thank you for your feedback: {feedback}. It has been stored successfully. Current Feedback History: {feedback_history}"

# Gradio interface
demo = gr.Blocks()

with demo:
    gr.Markdown("# AI Recovery Score & Plan Generator with Feedback")
    
    with gr.Row():
        with gr.Column():
            heart_rate = gr.Number(label="Heart Rate")
            bp = gr.Textbox(label="Blood Pressure")
            oxygen_saturation = gr.Textbox(label="Oxygen Saturation")
            symptoms = gr.Textbox(label="Symptoms")
            feedback = gr.Textbox(label="Feedback", visible=False)  # Initially hidden
            submit = gr.Button("Submit Vitals")
            feedback_button = gr.Button("Submit Feedback", visible=False)  # Initially hidden

        with gr.Column():
            recovery_score = gr.Textbox(label="Predicted Recovery Score", interactive=False)
            recovery_plan = gr.Textbox(label="Recovery Plan", interactive=False)
            feedback_prompt = gr.Textbox(label="Feedback Prompt", interactive=False)

    # Define actions
    submit.click(
        process_input,
        inputs=[heart_rate, bp, oxygen_saturation, symptoms],
        outputs=[recovery_score, recovery_plan, feedback_prompt],
    ).then(
        lambda: (gr.update(visible=True), gr.update(visible=True)),
        outputs=[feedback, feedback_button],
    )
    
    feedback_button.click(
        process_feedback,
        inputs=[heart_rate, bp, oxygen_saturation, symptoms, feedback],
        outputs=[feedback_prompt],
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)